# Storage

## volumes

In [ ]:
# ¿Que volumenes tengo actualmente en mi maquina?
!docker volume ls

In [8]:
# ¿Donde están?
# Terminal
# !cd /var/lib/docker/volumes/

/bin/sh: line 0: cd: /var/lib/docker/volumes/: No such file or directory


In [9]:
# Creamos uno nuevo
!docker volume create my-data-1

my-data-1


In [10]:
# Se puede ver en /var/lib/docker/volumes/
!docker volume ls

DRIVER              VOLUME NAME
local               my-data-1


In [11]:
# Algo de info del volumen
!docker volume inspect my-data-1

[
    {
        "CreatedAt": "2020-06-23T11:04:18Z",
        "Driver": "local",
        "Labels": {},
        "Mountpoint": "/var/lib/docker/volumes/my-data-1/_data",
        "Name": "my-data-1",
        "Options": {},
        "Scope": "local"
    }
]


In [12]:
# Eliminamos el volumen
!docker volume rm my-data-1

my-data-1


In [ ]:
# Arranquemos un ubuntu en modo interactivo con un volumen montado
# Terminal
!docker run -it -v my-data:/my-data ubuntu bash

Podemos crear un archivo en el volumen, matar el contenedor y crear otro con el mismo volumen montado

## bind mounts

In [ ]:
# Terminal
!docker run -it -v "$(pwd)"/host-data:/host-data ubuntu bash 

¿Que significa `"$(pwd)"`?
> Print working directory

Podemos modificar el archivo en el Host y vamos a ver los cambios en el contenedor
También podemos crear archivos en el contenedor y verlos en el Host

# Docker Compose

Extendamos la aplicación de la clase anterior y agreguemosle un contador en memoria de la cantidad de visitas.  
Vamos a usar una base de datos en memoria llamada Redis http://try.redis.io/

In [2]:
!pwd

/Users/florenciasilvestre/Documents/Flor/projects/AWS-Cloud-Practices/08-ECS


In [3]:
!cat flask-redis/app.py

#!/usr/bin/env python3
import os
from flask import Flask, jsonify
import redis

app = Flask(__name__)
cache = redis.Redis(host='redis', port=6379)


@app.route("/")
def hello():
	try:
		visits = cache.incr("counter")
	except redis.RedisError:
		print('Cant connect to Redis')
		visits = "NaN"

	return jsonify(message=f"Hello! I'm inside a container. You have called me {visits} times. ")


if __name__ == '__main__':
	app.run(host='0.0.0.0', debug=True)


In [4]:
# Hay una nueva dependencia
!cat flask-redis/requirements.txt

Flask
Redis

In [5]:
# ¿Cual seria el equivalente en docker run?
!cat flask-redis/docker-compose.yml

version: '3'
services:
  web:
    build: ""
    ports:
      - "80:5000"
  redis:
    image: "redis:alpine"

In [ ]:
# Levantamos todos los containers que haya en el archivo `docker-compose.yml` de esta carpeta
# Terminal
# !cd flask-redis && docker-compose up

In [20]:
!curl http://localhost

curl: (7) Failed to connect to localhost port 80: Connection refused


In [8]:
# Imagenes construidas
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
flask-redis_web     latest              8017c4fbecea        23 seconds ago      924MB
ubuntu              latest              74435f89ab78        6 days ago          73.9MB
redis               alpine              b546e82a6d0e        13 days ago         31.5MB
python              3.6                 13efce2de907        13 days ago         914MB


In [9]:
# Imagenes utilizadas por este cluster
!cd flask-redis && docker-compose images

     Container          Repository       Tag       Image Id       Size  
------------------------------------------------------------------------
flask-redis_redis_1   redis             alpine   b546e82a6d0e   31.51 MB
flask-redis_web_1     flask-redis_web   latest   8017c4fbecea   924.2 MB


In [66]:
# Rebuildear las imagenes en caso de que hagamos algun cambio
#!cd flask-redis && docker-compose build

redis uses an image, skipping
Building web
Step 1/6 : FROM python:3.6
 ---> 13efce2de907
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> 9f6727b6e674
Step 3/6 : COPY . /app
 ---> Using cache
 ---> cfe17c40b6f3
Step 4/6 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 05d73edda376
Step 5/6 : EXPOSE 5000
 ---> Using cache
 ---> e347394d234e
Step 6/6 : CMD ["python", "app.py"]
 ---> Using cache
 ---> 2580bf21145f
Successfully built 2580bf21145f
Successfully tagged flask-redis_web:latest


In [10]:
# Docker Compose crea un DNS interno para que podamos acceder a los otros contenedores por su nombre
!cd flask-redis && docker-compose config

services:
  redis:
    image: redis:alpine
  web:
    build:
      context: /Users/florenciasilvestre/Documents/Flor/projects/AWS-Cloud-Practices/08-ECS/flask-redis
    ports:
    - 80:5000/tcp
version: '3.0'



In [11]:
# ¿Como está nuestro cluster?
!cd flask-redis && docker-compose ps

       Name                     Command             State          Ports        
--------------------------------------------------------------------------------
flask-redis_redis_1   docker-entrypoint.sh redis    Up      6379/tcp            
                      ...                                                       
flask-redis_web_1     python app.py                 Up      0.0.0.0:80->5000/tcp


In [12]:
# Matemos el Redis
!cd flask-redis && docker-compose stop redis

Stopping flask-redis_redis_1 ... 
ping flask-redis_redis_1 ... done

In [13]:
# Se murio el contador
!curl http://localhost

{
  "message": "Hello! I'm inside a container. You have called me NaN times. "
}


In [14]:
# Resucitemos el contador
!cd flask-redis && docker-compose start redis

Starting redis ... 
ting redis ... done

In [24]:
# Volvemos a ver el contador
!curl http://localhost

curl: (7) Failed to connect to localhost port 80: Connection refused


In [16]:
# Paramos todos los servicios
!cd flask-redis && docker-compose stop

Stopping flask-redis_redis_1 ... 
Stopping flask-redis_web_1   ... 
ping flask-redis_web_1   ... done

In [17]:
# Construimos la imagen sola
!docker build -t flask-redis flask-redis

Sending build context to Docker daemon  10.75kB
Step 1/6 : FROM python:3.6
 ---> 13efce2de907
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> 1a43efcb9bb4
Step 3/6 : COPY . /app
 ---> 54635d9c5b12
Step 4/6 : RUN pip install -r requirements.txt
 ---> Running in 6c470ef92bad
Removing intermediate container 6c470ef92bad
 ---> 86b620369eb2
Step 5/6 : EXPOSE 5000
 ---> Running in 80983d91b272
Removing intermediate container 80983d91b272
 ---> 231c783288f3
Step 6/6 : CMD ["python", "app.py"]
 ---> Running in 74a719cc2ef3
Removing intermediate container 74a719cc2ef3
 ---> 329998504fbc
Successfully built 329998504fbc
Successfully tagged flask-redis:latest


In [18]:
# Taggeamos y pusheamos porque lo vamos a usar mas adelante
!docker tag flask-redis fsilvestre/flask-redis

In [19]:
!docker push fsilvestre/flask-redis


The push refers to repository [docker.io/fsilvestre/flask-redis]

a335dbc8: Preparing 
1ac6331d: Preparing 
463ce1ff: Preparing 
703d6103: Preparing 
f6aa0d01: Preparing 
b9f75f56: Preparing 
305e9fd3: Preparing 
18a2e1b1: Preparing 
4ca91984: Preparing 
de4639e0: Preparing 
89d2d3c5: Preparing 
335dbc8: Pushed   10.81MB/10.09MBflask [2Klatest: digest: sha256:564b1a380246f3d31ce99cb9345db7a470e2c7f5fbf6fdfe64c900f3af5829ba size: 2843
